In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
import os.path
import pickle

In [2]:
train = pd.read_pickle('train_pickle.pkl');
test = pd.read_pickle('test_pickle.pkl');

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_final
0,0000997932d777bf,"[explanation, why, the, edits, made, under, my...",0,0,0,0,0,0,"[explanation, edits, make, username, hardcore,..."
1,000103f0d9cfb60f,"[d'aww, !, he, matches, this, background, colo...",0,0,0,0,0,0,"[match, background, colour, seemingly, stick, ..."
2,000113f07ec002fd,"[hey, man, ,, i, 'm, really, not, trying, to, ...",0,0,0,0,0,0,"[hey, man, really, try, edit, war, guy, consta..."
3,0001b41b1c6bb37e,"[``, more, i, ca, n't, make, any, real, sugges...",0,0,0,0,0,0,"[ca, make, real, suggestion, improvement, wond..."
4,0001d958c54c6e35,"[you, ,, sir, ,, are, my, hero, ., any, chance...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]"


In [4]:
test.head()

,id,comment_text,comment_text_final
0,00001cee341fdb12,"[yo, bitch, ja, rule, is, more, succesful, the...","[yo, bitch, ja, rule, succesful, ever, whats, ..."
1,0000247867823ef7,"[==, from, rfc, ==, the, title, is, fine, as, ...","[rfc, title, fine, imo]"
2,00013b17ad220c46,"[``, ==, sources, ==, *, zawe, ashton, on, lap...","[source, zawe, ashton, lapland]"
3,00017563c3f7919a,"[:, if, you, have, a, look, back, at, the, sou...","[look, back, source, information, update, corr..."
4,00017695ad8997eb,"[i, do, n't, anonymously, edit, articles, at, ...","[anonymously, edit, article]"


In [5]:
vec = TfidfVectorizer(ngram_range=(1, 2), min_df=3,
                      max_df=0.9, strip_accents='unicode', use_idf=1, smooth_idf=1, sublinear_tf=1)
train['comment_text_final'] = [" ".join(text) for text in train['comment_text_final'].values]
vec = vec.fit(train['comment_text_final'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train['comment_text_final'], 
                                                    train['toxic'], test_size=0.33, random_state=42)

In [7]:
train_term_doc = vec.transform(X_train)
test_term_doc = vec.transform(X_test)

In [16]:
scaler = MaxAbsScaler()
train_term_doc = scaler.fit_transform(train_term_doc)
test_term_doc = scaler.fit_transform(test_term_doc)

In [17]:
if os.path.isfile('svm.sav'):
    clf = pickle.load(open('svm.sav', 'rb'))
else:
    clf = svm.SVC(gamma='scale', verbose=1)
    clf.fit(train_term_doc, y_train)
    pickle.dump(clf, open('svm.sav', 'wb'))

[LibSVM]

In [18]:
f1_score(y_test, clf.predict(test_term_doc))

0.599676724137931